## Data for TDS Eurovision: Migration.

Data pipeline to get migration data for countries participating in the Eurovision song contest, normalised by population size.
A couple of different source data files have been merged for this.

### Raw data description
#### 1) Migration
File: migration-flows.csv
- Data from [OWID](https://ourworldindata.org/migration) on international migration, section *Explore data on where people migrate from and to*.
- Original source is from the [UN](https://www.un.org/development/desa/pd/content/international-migrant-stock).
- Data shows total number of immigrants in each country split by country of origin in years 1990-2020 every 5 years.

### 2) Country codes (2 letters + Country name)
File: country_codes.xlsx
- From SharePoint (Katriona)

### 3) Voting data
File: final_voting_data_long.xlsx
- From SharePoint (Katriona)

### 4) Country codes (2 and 3 letters codes)
File: Table scraped from (iban.com)[https://www.iban.com/country-codes] (see below). Needed to join population size data.

### 5) Population sizes
File: pop_sizes.csv
- From the [**worldbank**](https://data.worldbank.org/indicator/SP.POP.TOTL?end=2021&start=2021&view=map). I selected pop sizes only for years 1990-2020 in 5 year steps to match migration data.

----------------------------------------------------------------------

## Data processing

In [1]:
import pandas as pd
import openpyxl as opx

### 1) Migration data

In [2]:
df_org = pd.read_csv('data/migration-flows.csv')

# overview
print(df_org.shape)
print(df_org.iloc[:, :7].head(3))

(1659, 484)
   Year      Country  Emigrants from Afghanistan  Immigrants to Afghanistan  \
0  1990  Afghanistan                  -7679582.0                    57686.0   
1  1995  Afghanistan                  -4347049.0                    71522.0   
2  2000  Afghanistan                  -4750677.0                    75917.0   

   Emigrants from Albania  Immigrants to Albania  Emigrants from Algeria  
0                     NaN                    NaN                     NaN  
1                     NaN                    NaN                     NaN  
2                     NaN                    NaN                     NaN  


Get into long format and clean data. Filter only Emigration, as we really want to know the nationalities of people in each country.

In [3]:
df = (df_org
    .pipe(pd.melt, id_vars=['Country', 'Year'], var_name='Migration', value_name='Count') # to long format
    .loc[lambda x: x['Migration'].str.contains('Emigrants')]                         # filter for emigrant rows
    .pipe(lambda x: x.rename(columns = {col: col.lower() for col in x.columns}))     # lowercase column names                                                         
    .assign(migration = lambda x: x.migration.str.replace('Emigrants from ', ''))    # filter for emigrant rows                          
    .rename(columns={'migration': 'emigrated_from'})                                 # boil down to country name
    .query('count >= 0')                                                             # negative counts are just total emigrants from country
    .pipe(lambda x: x.assign(count = x['count'].astype(int)))                        # convert count to int     
)
df

,country,year,emigrated_from,count
56,Argentina,1990,Afghanistan,20
57,Argentina,1995,Afghanistan,20
58,Argentina,2000,Afghanistan,20
59,Argentina,2005,Afghanistan,16
60,Argentina,2010,Afghanistan,9
...,...,...,...,...
784695,Zambia,2000,Zimbabwe,9056
784696,Zambia,2005,Zimbabwe,12444
784697,Zambia,2010,Zimbabwe,17226
784698,Zambia,2015,Zimbabwe,13175


Just a check

In [4]:
df[(df['country'] == 'Germany') & (df['emigrated_from'] == 'Turkey')].head(3)

,country,year,emigrated_from,count
717241,Germany,1990,Turkey,1459167
717242,Germany,1995,Turkey,1617880
717243,Germany,2000,Turkey,1604965


### 2) Country codes

To both country and emigrated_from variables.

In [5]:
# load country codes
cc = (pd.read_excel('data/country_codes.xlsx')
        .rename(columns={'country_name': 'country'})
        .drop(['country_border_code', 'country_border_name'], axis=1)
        .drop_duplicates())
cc

,country_code,country
0,AD,Andorra
2,AE,United Arab Emirates
4,AF,Afghanistan
10,AG,Antigua and Barbuda
11,AI,Anguilla
...,...,...
707,YE,Yemen
709,YT,Mayotte
710,ZA,South Africa
716,ZM,Zambia


A few country names don't match between datasets. Change them manually.

In [6]:
cc.loc[cc['country'] == 'United Kingdom of Great Britain and Northern Ireland', 'country'] = 'United Kingdom'
cc.loc[cc['country'] == 'Russian Federation', 'country'] = 'Russia'
cc.loc[cc['country'] == 'Moldova (the Republic of)', 'country'] = 'Moldova'

Add country codes to migration data, once for the focal country and once for the emigrated_from country.

In [7]:
df = (df
    .merge(cc, on='country', how='left')
    .merge(cc, left_on='emigrated_from', right_on='country', how='left')
    .drop('country_y', axis=1)
    .rename(columns={'country_x': 'country', 'country_code_x': 'code', 'country_code_y': 'emigrated_from_code'})
    .reindex(columns=['country', 'code', 'year', 'emigrated_from', 'emigrated_from_code', 'count']))


Another check

In [8]:
df[(df['country'] == 'Germany') & (df['emigrated_from'] == 'Turkey')]

,country,code,year,emigrated_from,emigrated_from_code,count
82873,Germany,DE,1990,Turkey,TR,1459167
82874,Germany,DE,1995,Turkey,TR,1617880
82875,Germany,DE,2000,Turkey,TR,1604965
82876,Germany,DE,2005,Turkey,TR,1315719
82877,Germany,DE,2010,Turkey,TR,1314223
82878,Germany,DE,2015,Turkey,TR,1343522
82879,Germany,DE,2020,Turkey,TR,1837282


### 3) Filter eurovision countries

In [9]:
vd = pd.read_excel('data/final_voting_data_long.xlsx')
# check whether GB or GBR or UK is in the 'From country' column
vd['From country'].unique()

array(['AT', 'BA', 'BE', 'CY', 'DE', 'DK', 'EE', 'ES', 'FR', 'GB', 'HR',
       'IE', 'IL', 'IS', 'LT', 'MT', 'NL', 'NO', 'PL', 'PT', 'SE', 'SI',
       'TR', 'CH', 'FI', 'LV', 'MK', 'RO', 'RU', 'GR', 'UA', 'AD', 'AL',
       'BY', 'MC', 'YU', 'BG', 'HU', 'MD', 'AM', 'CZ', 'GE', 'ME', 'RS',
       'AZ', 'SM', 'SK', 'IT', 'AU'], dtype=object)

In [10]:
# get set of all country codes in "From country" and "To country" columns
codes = set(vd['From country'].unique()).union(set(vd['To country'].unique()))
len(codes)

49

In [11]:
# which code in codes is not in the code variable in df? -> yugoslawia
codes.difference(set(df['code'].unique()))

{'YU'}

Summarise all non eurovision countries in one code 'Other'.

In [12]:
df = (df
    .pipe(lambda x: x.assign( # replace all non-Eurovision codes with 'Other'
        code = x['code'].apply(lambda x: x if x in codes else 'Other'),
        emigrated_from_code = x['emigrated_from_code'].apply(lambda x: x if x in codes else 'Other'),
        emigrated_from = x['emigrated_from'].apply(lambda x: 'Other' if x == 'Other' else x)))
    # filter out 'Other' from country and aggregate the rest to compress 'Other' into one row per country
    .query('code != "Other"') # remove all rows with 'Other' as code
    .groupby(['country', 'code', 'year', 'emigrated_from_code']) 
    .agg({'count': 'sum'})
    .reset_index()
)

In [13]:
# check again
print(df.shape)
df[df['country'] == 'Germany'].head(10)

#df.query('emigrated_from_code == "Other"').head(20)

(12649, 5)


,country,code,year,emigrated_from_code,count
4382,Germany,DE,1990,AL,5211
4383,Germany,DE,1990,AM,5581
4384,Germany,DE,1990,AT,189349
4385,Germany,DE,1990,AU,6205
4386,Germany,DE,1990,AZ,600
4387,Germany,DE,1990,BA,108349
4388,Germany,DE,1990,BE,21902
4389,Germany,DE,1990,BG,27077
4390,Germany,DE,1990,BY,1190
4391,Germany,DE,1990,CH,33253


## 4) add population size for each country


First need 3 letter country codes to join the population size table and the migration table.

In [14]:
# get table from IBAN website
tables = pd.read_html("https://www.iban.com/country-codes")
codes = tables[0]

Clean up.

In [18]:
codes = (codes
    .pipe(lambda x: x.rename(columns = {col: col.lower() for col in x.columns}))
    .rename(columns={'alpha-2 code': 'code', 'alpha-3 code': 'code3'})
    .drop(['country', 'numeric'], axis=1)
)

Add three letter country codes to migration data.

In [19]:
df = df.merge(codes, on='code', how='left')
df.head(3)

,country,code,year,emigrated_from_code,count,code3
0,Albania,AL,1990,GR,40087,ALB
1,Albania,AL,1990,IT,11287,ALB
2,Albania,AL,1990,MK,678,ALB


Population size data from the [**worldbank**](https://data.worldbank.org/indicator/SP.POP.TOTL?end=2021&start=2021&view=map). I selected pop sizes only for years 1990-2020 in 5 year steps to match migration data.

In [40]:
pop_size = (pd.read_csv('data/pop_sizes.csv')
           .iloc[:, 3:]
           .rename(columns=lambda x: x.lower().replace(' ', '_'))
           .pipe(pd.melt, id_vars=['country_code'], var_name='year', value_name='population')
           .assign(year=lambda x: x['year'].apply(lambda y: y.split('_')[0]))
           .assign(year=lambda x: x['year'].astype(int))
           .rename(columns={'country_code': 'code3'})
           .dropna()
           .assign(population=lambda x: pd.to_numeric(x['population'], errors='coerce'))
           )

Add population size to migration data.

In [41]:
df = (df.merge(pop_size, on=['year', 'code3'], how='left')
      .assign(prop_emigrants=lambda x: x['count'] / x['population'])
      .reindex(columns=['country', 'code', 'code3', 'population', 'year', 'emigrated_from_code', 'count', 'prop_emigrants'])
      )

,country,code,code3,population,year,emigrated_from_code,count,prop_emigrants
0,Albania,AL,ALB,3286542.0,1990,GR,40087,0.012197
1,Albania,AL,ALB,3286542.0,1990,IT,11287,0.003434
2,Albania,AL,ALB,3286542.0,1990,MK,678,0.000206
3,Albania,AL,ALB,3286542.0,1990,Other,4162,0.001266
4,Albania,AL,ALB,3286542.0,1990,TR,2489,0.000757
...,...,...,...,...,...,...,...,...
12644,United Kingdom,GB,GBR,67081000.0,2020,SE,35824,0.000534
12645,United Kingdom,GB,GBR,67081000.0,2020,SI,3668,0.000055
12646,United Kingdom,GB,GBR,67081000.0,2020,SK,72209,0.001076
12647,United Kingdom,GB,GBR,67081000.0,2020,TR,97205,0.001449


In [42]:
# save to csv
df.to_csv('data/migration.csv', index=False)